# Tarea I

In [183]:
# importacion de bibliotecas
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [184]:
# llamamos los archivos csv y json
licencias_pd = pd.read_csv('Licencias_Locales_202104.csv', delimiter=';')
terrazas_pd = pd.read_csv('Terrazas_202104.csv', delimiter=';', encoding='ISO-8859-1')
locales_pd = pd.read_csv('Locales_202104.csv', delimiter=';', encoding='ISO-8859-1')
books_pd = pd.read_json('books_edit_corrected.json')


In [185]:
# eliminar columnas con mas del 50% de datos nulos
licencias_copy = licencias_pd.copy()
terrazas_copy = terrazas_pd.copy()
locales_copy = locales_pd.copy()
books_copy = books_pd.copy()
print("dimension archivo licencias.csv: ",licencias_copy.shape)
print("dimension archivo terrazas.csv: ",terrazas_copy.shape)
print("dimension archivo locales.csv: ",locales_copy.shape)
print("dimension archivo books.json: ",books_copy.shape)
datasets = [licencias_copy, terrazas_copy, locales_copy, books_copy]

for dataset in datasets:
    threshold = len(dataset) * 0.5
    dataset.dropna(thresh=threshold, axis=1, inplace=True)

print("\ndimension archivo licencias.csv editado: ",licencias_copy.shape)
print("dimension archivo terrazas.csv editado: ",terrazas_copy.shape)
print("dimension archivo locales.csv editado: ",locales_copy.shape)
print("dimension archivo books.json editado: ",books_copy.shape)


dimension archivo licencias.csv:  (132171, 46)
dimension archivo terrazas.csv:  (6275, 57)
dimension archivo locales.csv:  (148216, 40)
dimension archivo books.json:  (431, 11)

dimension archivo licencias.csv editado:  (132171, 43)
dimension archivo terrazas.csv editado:  (6275, 53)
dimension archivo locales.csv editado:  (148216, 28)
dimension archivo books.json editado:  (431, 11)


In [186]:
terrazas_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6275 entries, 0 to 6274
Data columns (total 53 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id_terraza                      6275 non-null   int64  
 1   id_local                        6275 non-null   int64  
 2   id_distrito_local               6275 non-null   int64  
 3   desc_distrito_local             6275 non-null   object 
 4   id_barrio_local                 6275 non-null   int64  
 5   desc_barrio_local               6275 non-null   object 
 6   id_ndp_edificio                 6275 non-null   int64  
 7   id_clase_ndp_edificio           6275 non-null   int64  
 8   id_vial_edificio                6275 non-null   int64  
 9   clase_vial_edificio             6275 non-null   object 
 10  desc_vial_edificio              6275 non-null   object 
 11  nom_edificio                    6275 non-null   object 
 12  num_edificio                    62

In [187]:
# estandarizar columnas numericas con StandardScaler
datasets = [licencias_copy, terrazas_copy, locales_copy, books_copy]

#cambiar decimal de ',' a '.'
terrazas_copy['Superficie_ES'] = terrazas_copy['Superficie_ES'].str.replace(',', '.')
terrazas_copy['Superficie_RA'] = terrazas_copy['Superficie_RA'].str.replace(',', '.')
# primero creamos en dataset Terrazas una columna Superficie_TO que es la suma de Superficie_ES y Superficie_RA
terrazas_copy['Superficie_ES'] = terrazas_copy['Superficie_ES'].astype(float)
terrazas_copy['Superficie_RA'] = terrazas_copy['Superficie_RA'].astype(float)
terrazas_copy['Superficie_TO'] = terrazas_copy['Superficie_ES'] + terrazas_copy['Superficie_RA']


for dataset in datasets:
    numeric_cols = dataset.select_dtypes(include=['float64', 'int64']).columns
     # Filtrar columnas que NO empiezan con "id"
    numeric_cols_to_scale = [col for col in numeric_cols if not col.startswith('id')]
    if numeric_cols_to_scale:
        scaler = StandardScaler()
        dataset[numeric_cols_to_scale] = scaler.fit_transform(dataset[numeric_cols_to_scale]).round(5)


In [188]:
# creacion columna Ratio de Superficie_TO/id_terraza en terrazas.csv
terrazas_copy['Ratio'] = terrazas_copy['Superficie_TO'] / terrazas_copy['id_terraza']
# guardamos el archivo editado
terrazas_copy.to_csv('Terrazas_Normalizadas.csv', index=False)

# Producto de num_acceso * id_tipo_licencia en licencias.csv
licencias_copy['Producto'] = licencias_copy['num_acceso'] * licencias_copy['id_tipo_licencia']

# Residuo desc_seccion_censal_local /num_edificio en locales.csv
locales_copy['Residuo'] = locales_copy['desc_seccion_censal_local'] % locales_copy['num_edificio']

# Tarea II


In [189]:
# eliminacion de duplicados en licencias.csv
licencias_copy.drop_duplicates(subset=['id_local','ref_licencia'],inplace=True)
licencias_copy.to_csv('Licencias_SinDuplicados.csv', index=False)

In [190]:
# pasar texto primera letra este en mayuscula y eliminar espacios en blanco
cols_string = ['title','isbn','thumbnailUrl','shortDescription','longDescription','status','authors','categories']

for col in cols_string:
    books_copy[col] = books_copy[col].astype(str).str.capitalize().str.strip().str.replace(r'\s+', ' ', regex=True)

# guardar archivo editado
books_copy.to_json('Books_Limpio.json', orient='records', lines=True)

# realizamos lo mismo con los otros archivos
datasets = [licencias_copy, terrazas_copy, locales_copy]

for dataset in datasets:
    cols_string = dataset.select_dtypes(include=['object']).columns
    for col in cols_string:
        dataset[col] = dataset[col].astype(str).str.capitalize().str.strip().str.replace(r'\s+', ' ', regex=True)



# Tarea III

In [191]:
# inner join de Terrazas_Normalizadas y Licencias_SinDuplicados con id_local

terrazas_normalizadas = pd.read_csv('Terrazas_Normalizadas.csv')
licencias_sinduplicados = pd.read_csv('Licencias_SinDuplicados.csv')
df_inner = pd.merge(terrazas_normalizadas, licencias_sinduplicados, on='id_local', how='inner')
df_inner['coordenada_x_local_x'] = df_inner['coordenada_x_local_x'].str.replace(',', '.')
df_inner['coordenada_y_local_x'] = df_inner['coordenada_y_local_x'].str.replace(',', '.')
df_inner['coordenada_x_local_y'] = df_inner['coordenada_x_local_y'].str.replace(',', '.')
df_inner['coordenada_y_local_y'] = df_inner['coordenada_y_local_y'].str.replace(',', '.')

df_inner.to_csv('Licencias_Terrazas_Integradas.csv', index=False,encoding='utf-8')


In [192]:
# Agregacion por barrio y usma de de superficies de terrazas del dataset terrazas.csv
terrazas_agg = terrazas_copy.groupby(['desc_barrio_local']).agg({'Superficie_TO':'sum', 'Superficie_ES':'sum','Superficie_RA':'sum'}).sort_values(by='Superficie_TO', ascending=False).round(2)
terrazas_agg.to_csv('Terrazas_Agregadas.csv')

In [193]:
locales_agg = locales_copy.groupby(
    ['desc_distrito_local']).agg({'Residuo':'sum'}
                                 ).sort_values(by='Residuo', ascending=False
                                               ).round(2)
locales_agg.to_csv('Locales_Agregados.csv')
licencias_agg = licencias_copy.groupby(
    ['desc_barrio_local', 'desc_tipo_situacion_licencia']
).size().reset_index(name='count').sort_values(by='count', ascending=False)
licencias_agg.to_csv('Licencias_Agregadas.csv', index=False)

# Tarea 4

In [194]:
# Concatenar datasets
# primero creamos una copia de licencias_locales y guardamos como Licencias_Locales_202105
licencias_copy.to_csv('Licencias_Locales_202105.csv', index=False)
licencias_copy_2 = pd.read_csv('Licencias_Locales_202105.csv')
# concatenamos los datasets
licencias_concat = pd.concat([licencias_copy, licencias_copy_2], axis=0)
licencias_concat_sin_duplicados = licencias_concat.drop_duplicates(subset=['id_local'])

In [195]:
terrazas_cols=['id_local','clase_vial_edificio','desc_vial_edificio','id_terraza','Superficie_TO','Ratio']
locales_cols=['id_local','desc_distrito_local','desc_barrio_local','desc_tipo_acceso_local','clase_vial_acceso','desc_vial_acceso','num_edificio','Residuo']
df_concat = pd.concat([terrazas_copy[terrazas_cols], locales_copy[locales_cols]], axis=1)
df_concat_sin_duplicados = df_concat.drop_duplicates(subset=['id_local'])